In [ ]:
#read in the full dataset

In [ ]:

#normalize the text
import re

def normalize_text(text):
    """
    Normalize the text by converting to lowercase, preserving commas and spaces,
    and removing unnecessary special characters.
    """
    if pd.isna(text):
        return ''
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    text = re.sub(r'[^\w\s,]', '', text)  # Remove all characters except alphanumeric, spaces, and commas
    return text.strip()  # Trim leading and trailing spaces
df['normalized_author'] = df['author'].apply(normalize_text)

In [ ]:
#get the VIAF entries for all data
def get_name(name):
    params = {
        'query' : f'local.personalNames = "{name}"',
        'maximumRecords': 10,
        'startRecord' : 1,
        'sortKeys': 'holdingscount',
        'httpAccept': 'application/json'
    }

    headers={'User-Agent': user_agent}
    url = "https://viaf.org/viaf/search"

    r = requests.get(url,params=params,headers=headers, timeout=10)
    data = r.json()

    return data

In [ ]:
user_agent = 'YOUR PROJECT NAME HERE'
id_column_name = "Name"

pause_between_req = 1

use_title_reconcilation = True

cache = {}

In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz

birthdates_df = df

# Function to perform search and return DataFrame with search results
def search_author(author_name):
    # Initialize list to store search results
    search_results = []

    try:
        data = get_name(author_name)
        records = data['searchRetrieveResponse']['records']
        total_records = len(records)

        for idx, record in enumerate(records):
            record_data = record['record']['recordData']
            birthdate = record['record']['recordData']['birthDate']
            viaf_title_list = []
            # Extract titles if available
            if 'titles' in record_data:
                titles_data = record_data['titles']
                if titles_data is not None:
                    if isinstance(titles_data['work'], list):
                        # Extract titles from the list of works
                        viaf_title_list.extend([work['title'] for work in titles_data['work'] if 'title' in work])
                    else:
                        # Extract title from a single work
                        title = titles_data['work'].get('title')
                        if title:
                            viaf_title_list.append(title)

#             # Get S2 titles for the author
#             s2_titles = birthdates_df.loc[birthdates_df['author'] == author_name, 'S2 Titles'].iloc[0]
#             if isinstance(s2_titles, float):
#                 s2_titles = []

#             # Perform fuzzy title matching
#             title_matched = False
#             matched_title = None
#             for title_from_viaf in title_list:
#                 for s2_title in s2_titles:
#                     if fuzz.partial_ratio(s2_title, title_from_viaf) >= 70:
#                         title_matched = True
#                         matched_title = s2_title
#                         break
#                 if title_matched:
#                     break

#             # Append author name, title list, S2 titles, title matched, matched title, and birthdate to search results
#             # birthdate = birthdates_df.loc[birthdates_df['author'] == author_name, 'birthdate'].iloc[0]
#             search_results.append({'author': author_name, 'record_count': len(records),
#                                    'record_enumerated': idx, 'title_list': title_list,
#                                    'S2 Titles': s2_titles, 'title_matched': title_matched,
#                                    'matched_title': matched_title, 'birthdate': birthdate})
            
            # Append author name, title list, S2 titles, title matched, matched title, and birthdate to search results
            # birthdate = birthdates_df.loc[birthdates_df['author'] == author_name, 'birthdate'].iloc[0]
            title_list = df.loc[df['author'] == author_name, 'title_list'].iloc[0]

            search_results.append({'author': author_name, 'record_count': len(records),
                                   'record_enumerated': idx, 'viaf_title_list': viaf_title_list,
                                   'birthdate': birthdate,'title_list':title_list})
    except Exception as e:
        print(f'Processing Error for {author_name}: {e}')

    return search_results

# Iterate through each author name in the original DataFrame and perform the search
all_search_results = []
row_counter = 0  # Counter to limit the number of rows processed

for author_name in birthdates_df['author']:
    search_results = search_author(author_name)
    all_search_results.extend(search_results)
    # row_counter += 1
    # if row_counter >= 5:  # Adjust the number as needed
    #     break

# Convert list of dictionaries to DataFrame
all_search_results_df = pd.DataFrame(all_search_results)
print(all_search_results_df)


In [ ]:
result_df = df
result_df['VIAF_titlelist'] = result_df['record_enumerated_titles'] 
result_df['S2_titlelist'] = result_df['S2titles']
result_df['average_S2_pubdate'] = result_df['avg_pubdates']
result_df['VIAF_birthdate'] = result_df['standard_birthdate']
result_df['S2_Author'] = result_df['author']


df = result_df

In [ ]:
def any_neg(birth2maxdate, birth2mindate):
    if birth2maxdate is not None and birth2mindate is not None:
        if birth2maxdate < 0 or birth2mindate < 0:
            return 1
    return 0

def title_list_len(title_list):
    return len(title_list) if isinstance(title_list, list) else 0

def author_length(author):
    return len(author)

In [ ]:
# df_test = df.head(10)
from ast import literal_eval

for idx, row in df.iterrows():
    birth = row['VIAF_birthdate']
    pubdates = row['S2_pubdates']
    titlelist = row['VIAF_titlelist']
    author = row['author']
    if pubdates is not None:
        print(pubdates)
        # print(birth)
        pubdates = literal_eval(pubdates)
        author = str(row['author'])
        birth2maxdate_value = max(pubdates) - int(birth)
        birth2mindate_value = min(pubdates)- int(birth)
        # any_neg = any_neg(birth2maxdate_value,birth2mindate_value)
        title_list_len = len(titlelist)
        author_len = len(author)
        
        # print(birth2maxdate_value)
        df.at[idx,'birth2max_value'] = birth2maxdate_value
        df.at[idx, 'birth2min_value'] = birth2mindate_value
        # df.at[idx, 'any_neg'] = any_neg
        df.at[idx, 'title_list_len'] = title_list_len
        df.at[idx, 'author_len'] = author_len
        # print(birth2maxdate_value)
    # else:
    #     print(pubdates, birth)
    


In [ ]:
import pandas as pd
import ast


# Function to compute average publication date
def compute_avg_pubdate(pubdates):
    try:
        # Convert the string representation to an actual tuple
        pubdates_tuple = ast.literal_eval(pubdates)
        
        # Check if it's a tuple of integers
        if isinstance(pubdates_tuple, tuple) and all(isinstance(date, int) for date in pubdates_tuple):
            return sum(pubdates_tuple) / len(pubdates_tuple)
        else:
            return None
    except (ValueError, SyntaxError):
        return None

# Apply the function to each row
df['avg_pubdate'] = df['S2_pubdates'].apply(lambda x: compute_avg_pubdate(str(x)))



In [ ]:
#publication_age
df['publication_age'] = df['avg_pubdate'] - df['VIAF_birthdate']

In [ ]:
df['publication_age'] = pd.to_numeric(df['publication_age'], errors='coerce')


In [ ]:
df = df.fillna(0)


In [ ]:
df['status'] = ""

for idx, row in df.iterrows():
    if row['publication_age'] < 0:
        df.at[idx,'status'] = 'not_born'
    elif row['publication_age'] > 100:
        df.at[idx,'status'] = 'zombie'
    elif row['publication_age'] < 9:
        df.at[idx,'status'] = 'toddler'
    else:
        df.at[idx,'status'] = '0'

In [ ]:
df['status'] = df['status'].str.replace('zombie', '1')
df['status'] = df['status'].fillna('0')
df['status'] = df['status'].str.replace('not_born', '2')
df['status'] = df['status'].str.replace('toddler', '3')


Now add VIAF and S2 titlelist embeddings

In [ ]:
#embeddings

from sentence_transformers import SentenceTransformer

# Load the pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Function to generate embeddings for a list of titles
def get_embeddings(titles_list):
    if isinstance(titles_list, str):
        return model.encode(titles_list)
    else:
        return model.encode('')

# Generate embeddings for each row in the DataFrame
df['S2_embeddings'] = df['S2titles'].apply(get_embeddings)
df['VIAF_embeddings'] = df['VIAF_titlelist'].apply(get_embeddings)


#was taking forever to run, so I'm leaving this piece out for now to make sure everything else works!

In [ ]:
import numpy as np
df['S2_embeddings'] = df['S2_embeddings'].apply(np.mean)
df['VIAF_embeddings'] = df['VIAF_embeddings'].apply(np.mean)

In [ ]:
#get overlapping words and overlapping lemmas:
# Function to lemmatize text and return a set of lemmatized words
import spacy
nlp = spacy.load("en_core_web_sm")

def lemmatize_text(text):
    doc = nlp(text)
    return set(token.lemma_ for token in doc if not token.is_stop and not token.is_punct)

# Function to calculate lemma overlap and return the overlapping lemmas
def calculate_lemma_overlap(text1, text2):
    text1 = str(text1)
    text2 = str(text2)
    lemmas1 = lemmatize_text(text1)
    lemmas2 = lemmatize_text(text2)
    overlap = lemmas1.intersection(lemmas2)
    return len(overlap), overlap

# # Apply the function to compute lemma overlap for each row in the DataFrame
# df[['lemma_overlap', 'overlapping_lemmas']] = df.apply(
#     lambda row: pd.Series(calculate_lemma_overlap(row['VIAF_titlelist'], row['S2_titlelist'])), 
#     axis=1)
df[['lemma_overlap', 'overlapping_lemmas']] = df.apply(
    lambda row: pd.Series(calculate_lemma_overlap(row['VIAF_titlelist'], row['S2_titlelist'])), 
    axis=1)


In [ ]:
#add overlaps

#add word overlap as a new feature
import pandas as pd
from nltk.corpus import stopwords
import nltk

# Download stopwords from NLTK if you haven't already
nltk.download('stopwords')
# Define stop words
stop_words = set(stopwords.words('english'))

# Function to create bag of words after removing stop words
def bag_of_words(title):
    words = title.lower().split()
    return set([word for word in words if word not in stop_words])

# Function to find and count word overlap
def find_word_overlap(row):
    try:
        v_list = ast.literal_eval(row['VIAF_titlelist'])
        s_list = ast.literal_eval(row['S2_titlelist'])
        if isinstance(v_list, list):

            for title in v_list:
                title = str(title)
        if isinstance(s_list, list):

            for title in s_list:
                title = str(title)
        if isinstance(v_list, list):

            v_bag = bag_of_words(v_list) 
        if isinstance(s_list, list):

            s_bag = bag_of_words(s_list) 

        overlap = v_bag & s_bag
    except:
        overlap = ''
    return pd.Series([len(overlap), list(overlap)])
  
# Apply the function to each row and create two new columns
df[['word_overlap_count', 'overlapping_words']] = df.apply(find_word_overlap, axis=1)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (precision_recall_curve, roc_auc_score,
                             confusion_matrix, ConfusionMatrixDisplay,
                             classification_report, accuracy_score)
import matplotlib.pyplot as plt


X = df.drop(columns=['author','VIAF_titlelist','S2_titlelist','overlapping_words','selected_birthyear','overlapping_lemmas','title_list','record_enumerated_titles','S2Titles','S2titles','matched_title_list','common_words','notes','normalized_author','S2_Titlelist','S2_Author','S2_pubdates','mean_embedding','matched_title?','match?','match'])  # Drop the label and metadata columns
y = df['match?']

# Optional: If you want to split into training and test sets first
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
# Initialize the Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Perform cross-validation
model = RandomForestClassifier()
y_pred_proba = cross_val_predict(model, X, y, cv=5, method='predict_proba')

# Store predictions with original indices
predictions_df = pd.DataFrame(y_pred_proba, index=original_indices, columns=['Class_0_Proba', 'Class_1_Proba'])

# Merge the metadata back
results_df = pd.concat([original_metadata, predictions_df], axis=1)
results_df['True_Label'] = y.loc[results_df.index]

# Now you have a DataFrame with metadata and predictions
print(results_df.head())

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import Binarizer

# Initialize the Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Perform cross-validation to get predicted probabilities
y_pred_proba = cross_val_predict(model, X, y, cv=5, method='predict_proba')

# Store predictions with original indices
predictions_df = pd.DataFrame(y_pred_proba, index=original_indices, columns=['Class_0_Proba', 'Class_1_Proba'])

# Merge the metadata back
results_df = pd.concat([original_metadata, predictions_df], axis=1)
results_df['True_Label'] = y.loc[results_df.index]

# Define the threshold for decision
threshold = 0.5  # You can change this value based on your needs

# Compute class decisions based on the threshold
results_df['Class_Decision'] = (results_df['Class_1_Proba'] > threshold).astype(int)

# Now you have a DataFrame with metadata, predictions, and class decisions
print(results_df.head())


In [ ]:
#find the optimal threshold
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report

# Initialize the Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Perform cross-validation to get predicted probabilities
y_pred_proba = cross_val_predict(model, X, y, cv=5, method='predict_proba')

# Store predictions with original indices
predictions_df = pd.DataFrame(y_pred_proba, index=original_indices, columns=['Class_0_Proba', 'Class_1_Proba'])

# Define the threshold for decision
threshold = 0.8  # You can change this value based on your needs

# Compute class decisions based on the threshold
predictions_df['Class_Decision'] = (predictions_df['Class_1_Proba'] > threshold).astype(int)

# Merge the metadata back
results_df = pd.concat([original_metadata, predictions_df], axis=1)
results_df['True_Label'] = y.loc[results_df.index]

# Generate class predictions
y_pred = (y_pred_proba[:, 1] > threshold).astype(int)  # Use the same threshold for predictions

# Print the DataFrame with metadata, predictions, and class decisions
print(results_df.head())

# Generate and print the classification report
report = classification_report(y, y_pred)
print("\nClassification Report:\n", report)
